In [1]:
import warnings
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import gc
import random
from tensorflow.keras import backend as K
from keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from config import CONFIG
import yaml
import ast
from kl_optimization import optimizeWeights
from model_utils import load_model

2025-03-24 23:05:29.058145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742879129.082147  177331 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742879129.086833  177331 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742879129.098125  177331 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742879129.098152  177331 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742879129.098154  177331 computation_placer.cc:177] computation placer alr

## Create Model Definition

In [2]:
def create_model(seed=None):
    """
    Creates and compiles a CNN model based on the provided configuration.

    Parameters:
    - config (dict): Configuration dictionary with keys:
        - 'random_seed': int
        - 'image_size': str (e.g., '(100, 100, 3)')
    
    Returns:
    - model (tf.keras.Model): Compiled Keras model
    """
    if not seed:
    # Set random seed for reproducibility
        seed = CONFIG.get('random_seed', 333)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
   
    # Parse image input shape from string
    input_shape = ast.literal_eval(CONFIG.get('image_size', '(100, 100, 3)'))
    # Build model
    model = Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(1, activation='sigmoid')
    ])

    # Compile model
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[
            tf.keras.metrics.Recall(),
            tf.keras.metrics.Precision(),
            tf.keras.metrics.AUC(
                num_thresholds=200,
                curve='ROC',
                summation_method='interpolation'
            ),
            'accuracy'
        ]
    )

    return model

## Train Model

### Load Training Data

In [3]:
def load_training_data(data_path):
    """
    Loads a processed .npy data file and returns feature and label arrays.

    Args:
        fold_path (str): Path to the .npy file (e.g., 'init.npy' or 'round1.npy')

    Returns:
        tuple: (trainx, trainy)
            - trainx (np.ndarray): Feature array of shape (N, H, W, C)
            - trainy (np.ndarray): Label array of shape (N, 1)
    """
    data = np.load(data_path, allow_pickle=True)
    trainx = np.stack(data[:, 0]).astype(np.float32)
    trainy = data[:, 1].astype(int).reshape([-1, 1])
    return trainx, trainy

### Model Training

In [4]:
def train_model(model, trainx, trainy, epochs=CONFIG['local_epochs'], batch_size=16, verbose=1, validation_data=None):
    """
    Trains a compiled Keras model.

    Args:
        model (tf.keras.Model): A compiled Keras model.
        trainx (np.ndarray): Training input data.
        trainy (np.ndarray): Training labels.
        epochs (int): Number of training epochs.
        batch_size (int): Size of training batches.
        verbose (int): Verbosity mode (0=silent, 1=progress bar, 2=one line per epoch).
        validation_data (tuple): Optional (val_x, val_y) for validation.

    Returns:
        tf.keras.callbacks.History: Training history object.
    """
    history = model.fit(
        trainx,
        trainy,
        epochs=epochs,
        batch_size=batch_size,
        verbose=verbose,
        validation_data=validation_data
    )
    return history

## Saving Model

In [5]:
def save_model(model, path):
    """
    Saves the given model to the specified path.

    Args:
        model (tf.keras.Model): The model to save.
        path (str): Path where the model will be saved.
    """
    model.save(path)
    print(f"💾 Model saved to: {path}")

## Model Deletion

In [6]:
def clear_model_from_memory(model):
    """
    Deletes the model from memory and clears GPU session.

    Args:
        model (tf.keras.Model): The model to delete.
    """
    del model
    K.clear_session()
    gc.collect()
    print("🧹 Model removed from memory and GPU session cleared.")

## Update Model with gradients

In [7]:
for i in range(3):
    model = create_model(1)
    trainx,trainy = load_training_data(f"../experiments/fed_ml_experiment_1/processed_data/client_1/round{i+1}.npy")
    train_model(model,trainx,trainy)
    save_model(model,f"../experiments/fed_ml_experiment_1/models/model+str{i+1}+.keras")
    clear_model_from_memory(model)

/home/yash/miniconda3/envs/fedml/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1742879134.709780  177331 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5582 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/10


I0000 00:00:1742879137.335357  177411 service.cc:152] XLA service 0x7f081800d170 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742879137.335412  177411 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2025-03-24 23:05:37.387959: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742879137.659414  177411 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/7 ━━━━━━━━━━━━━━━━━━━━ 28s 5s/step - accuracy: 0.3125 - auc: 0.4048 - loss: 0.7157 - precision: 0.0000e+00 - recall: 0.0000e+00

I0000 00:00:1742879140.499682  177411 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 676ms/step - accuracy: 0.5133 - auc: 0.5978 - loss: 0.7389 - precision: 0.5276 - recall: 0.6497
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7084 - auc: 0.9153 - loss: 0.4988 - precision: 0.6915 - recall: 0.9608
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9050 - auc: 0.9479 - loss: 0.2735 - precision: 0.9477 - recall: 0.8901
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8692 - auc: 0.9348 - loss: 0.3774 - precision: 0.9095 - recall: 0.8750
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9193 - auc: 0.9451 - loss: 0.2660 - precision: 0.9416 - recall: 0.9244
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8687 - auc: 0.9410 - loss: 0.2802 - precision: 0.9598 - recall: 0.8166
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9394 - auc: 0.9561 - loss: 0.2054 - precision: 0.9858 - recall: 0.9120
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9471 - au

In [8]:
testx,testy = load_training_data('../experiments/fed_ml_experiment_1/processed_data/client_5/round3.npy')
preds = []
for i in range(3):
    model = load_model(f"../experiments/fed_ml_experiment_1/models/model+str{i+1}+.keras")
    preds.append(model.predict(testx))
    clear_model_from_memory(model)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step
🧹 Model removed from memory and GPU session cleared.
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step
🧹 Model removed from memory and GPU session cleared.
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/stepWARNING:tensorflow:6 out of the last 12 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7f08645f11c0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step
🧹 Model removed fr

In [9]:
model = load_model(f"../experiments/fed_ml_experiment_1/models/model+str{1}+.keras")
model.weights

[<Variable path=conv2d/kernel, shape=(3, 3, 3, 32), dtype=float32, value=[[[[-7.35872537e-02  9.23789963e-02  1.05603151e-01 -3.10195908e-02
     -1.41198292e-01 -1.08481914e-01  1.38706071e-02 -1.31445646e-01
      7.40582496e-02 -6.61253780e-02  2.73669660e-02 -1.00024849e-01
      3.66791114e-02 -5.35564013e-02  2.54453216e-02 -1.22777551e-01
     -6.27939403e-02 -2.80196569e-03  1.22878835e-01 -5.65728620e-02
     -1.03071257e-01 -3.47342193e-02  3.58970948e-02  1.15068495e-01
      5.12849279e-02 -6.21851683e-02  9.91087258e-02 -5.50943576e-02
     -8.97516757e-02 -9.34584886e-02 -1.16810098e-01  8.45249668e-02]
    [ 4.16899845e-03 -1.30309552e-01 -1.43145427e-01 -3.15809622e-02
     -4.04535457e-02  2.72590518e-02  9.87870321e-02  6.75289482e-02
      7.54912943e-02 -4.50992212e-02 -5.42165451e-02  1.07773706e-01
     -2.23069731e-02 -4.25663292e-02  5.92814712e-03  5.83368801e-02
      1.08592302e-01  1.18047625e-01 -3.87738906e-02  9.68478546e-02
      5.70863150e-02 -6.749708

In [10]:
teachers = preds[1:]
optimizeWeights(model,testx,testy,teachers)
model.weights

Epoch 000: Loss: 0.250, Accuracy: 87.755%
Epoch 001: Loss: 0.248, Accuracy: 93.878%
Epoch 002: Loss: 0.209, Accuracy: 89.796%
Epoch 003: Loss: 0.128, Accuracy: 95.918%
Epoch 004: Loss: 0.114, Accuracy: 95.918%
Epoch 005: Loss: 0.072, Accuracy: 97.959%
Epoch 006: Loss: 0.060, Accuracy: 98.980%
Epoch 007: Loss: 0.061, Accuracy: 97.959%
Epoch 008: Loss: 0.049, Accuracy: 97.959%
Epoch 009: Loss: 0.051, Accuracy: 100.000%


[<Variable path=conv2d/kernel, shape=(3, 3, 3, 32), dtype=float32, value=[[[[-0.07606848  0.08917874  0.10722977 -0.03794298 -0.13984676
     -0.11054628  0.00638723 -0.1339565   0.07006491 -0.06382864
      0.01919606 -0.09495902  0.0431094  -0.05104288  0.03548045
     -0.12870297 -0.06437021 -0.00764872  0.12411702 -0.06124344
     -0.10621083 -0.03045037  0.03693702  0.11625874  0.05082476
     -0.05752266  0.09553943 -0.06493816 -0.08775091 -0.09825826
     -0.12021067  0.08203184]
    [ 0.00325476 -0.13352044 -0.14436188 -0.0376126  -0.04049645
      0.02750403  0.08802013  0.06669807  0.07348514 -0.04213434
     -0.06250498  0.11322401 -0.01008711 -0.03764682  0.01369073
      0.0507802   0.1078998   0.11524435 -0.03841388  0.09175528
      0.05420363 -0.06865551 -0.02385405  0.05858213 -0.06653298
      0.04632099 -0.12411261 -0.05698922  0.00835628  0.00752159
     -0.122233    0.09610622]
    [-0.0286999  -0.15355703  0.02330925 -0.00196248  0.02699946
      0.14105462 -0.043

In [22]:
del model
K.clear_session()
gc.collect()

0